# Taxonomic Profiling using Kraken2

We first need to download and install a reference database. There are a number of pre-computed databases here: https://benlangmead.github.io/aws-indexes/k2 . For demonstration purposes, let's use one of the smallest ones : Standard-8. This is available in  `data/kraken2_db/`

First, create a folder where the kraken2 outputs go.

In [1]:
! mkdir -p results/kraken2_output

To run kraken on one sample, you would do :

`kraken2 --paired --db data/kraken2_db/standard-8/ data/metagenome_samples/FH1_1.fastq.gz data/metagenome_samples/FH1_2.fastq.gz --output results/kraken2_output/FH1`

To run on all at once, use the script below.

In [2]:
!for sample in $(ls data/metagenome_samples/*_1.fastq.gz | sed 's/.*\///' | sed 's/_1.*//'); do \
    echo "Processing sample: ${sample}"; \
    kraken2 --paired --db data/kraken2_db/standard-8/  --output results/kraken2_output/${sample}.kraken.output --report results/kraken2_output/${sample}.kraken.report \
    data/metagenome_samples/${sample}_1.fastq.gz data/metagenome_samples/${sample}_2.fastq.gz;\
done

Processing sample: BH1
Loading database information... done.
1000000 sequences (302.00 Mbp) processed in 26.460s (2267.6 Kseq/m, 684.81 Mbp/m).
  705810 sequences classified (70.58%)
  294190 sequences unclassified (29.42%)
Processing sample: BH2
Loading database information... done.
1000000 sequences (302.00 Mbp) processed in 33.129s (1811.1 Kseq/m, 546.94 Mbp/m).
  336398 sequences classified (33.64%)
  663602 sequences unclassified (66.36%)
Processing sample: BH3
Loading database information... done.
1000000 sequences (302.00 Mbp) processed in 43.397s (1382.6 Kseq/m, 417.54 Mbp/m).
  208838 sequences classified (20.88%)
  791162 sequences unclassified (79.12%)
Processing sample: BH4
Loading database information... done.
1000000 sequences (302.00 Mbp) processed in 41.553s (1443.9 Kseq/m, 436.07 Mbp/m).
  261480 sequences classified (26.15%)
  738520 sequences unclassified (73.85%)
Processing sample: FH1
Loading database information... done.
1000000 sequences (302.00 Mbp) processed in

In [10]:
!head  results/kraken2_output/BH1.report 


 29.42	294190	294190	U	0	unclassified
 70.58	705810	485	R	1	root
 70.50	704961	2859	R1	131567	  cellular organisms
 68.09	680862	29099	D	2	    Bacteria
 63.41	634069	38696	P	1224	      Pseudomonadota
 55.35	553477	2461	C	28216	        Betaproteobacteria
 54.62	546158	9330	O	80840	          Burkholderiales
 51.18	511791	12223	F	506	            Alcaligenaceae
 49.69	496927	34939	G	222	              Achromobacter
 44.69	446934	1893	G1	2626865	                unclassified Achromobacter


bracken -d data/kraken2_db/standard-8/ -l G -i results/kraken2_output/BH1.report -o results/kraken2_output/BH1.bracken.output

In [6]:
!for sample in $(ls data/metagenome_samples/*_1.fastq.gz | sed 's/.*\///' | sed 's/_1.*//'); do \
    bracken -d data/kraken2_db/standard-8/ -l G -i results/kraken2_output/${sample}.kraken.report -r 100 -o results/kraken2_output/${sample}.bracken.output;\
done

 >> Checking for Valid Options...
 >> Running Bracken 
      >> python src/est_abundance.py -i results/kraken2_output/BH1.kraken.report -o results/kraken2_output/BH1.bracken.output -k data/kraken2_db/standard-8/database100mers.kmer_distrib -l G -t 0
PROGRAM START TIME: 01-17-2025 02:26:48
>> Checking report file: results/kraken2_output/BH1.kraken.report
BRACKEN SUMMARY (Kraken report: results/kraken2_output/BH1.kraken.report)
    >>> Threshold: 0 
    >>> Number of genuses in sample: 1118 
	  >> Number of genuses with reads > threshold: 1118 
	  >> Number of genuses with reads < threshold: 0 
    >>> Total reads in sample: 1000000
	  >> Total reads kept at genuses level (reads > threshold): 598537
	  >> Total reads discarded (genuses reads < threshold): 0
	  >> Reads distributed: 107257
	  >> Reads not distributed (eg. no genuses above threshold): 16
	  >> Unclassified reads: 294190
BRACKEN OUTPUT PRODUCED: results/kraken2_output/BH1.bracken.output
PROGRAM END TIME: 01-17-2025 02:26:49

We could just proceed with the output file, but so that we can reuse the statistical analysis script used for metaphlan, we will convert bracken outout to match the metaphlan format.

In [9]:
!for sample in $(ls data/metagenome_samples/*_1.fastq.gz | sed 's/.*\///' | sed 's/_1.*//'); do \
    kreport2mpa.py -r results/kraken2_output/${sample}.kraken_bracken_genuses.report -o results/kraken2_output/${sample}_bracken.mpa; \
done 

In [13]:
!for sample in $(ls data/metagenome_samples/*_1.fastq.gz | sed 's/.*\///' | sed 's/_1.*//'); do \
    rm -f results/kraken2_output/${sample}_abundance.mpa;\
    sum=$(grep -vP "\\|" results/kraken2_output/${sample}_bracken.mpa | cut -f 2 | awk '{sum += $1} END {printf "%.2f", sum/100}'); \
    awk -v sum="$sum" 'BEGIN {FS="\t"; OFS="\t"} {print $1, $2/sum}' results/kraken2_output/${sample}_bracken.mpa >> results/kraken2_output/${sample}_abundance.mpa;\
done

Merge 

In [15]:
! rm -f results/kraken2_output/merged_kraken_abundance_table.txt
! merge_metaphlan_tables.py results/kraken2_output/*abundance.mpa >> results/kraken2_output/merged_kraken_abundance_table.txt

In [16]:
! sed -i '1s/_abundance//g' results/kraken2_output/merged_kraken_abundance_table.txt